# GOAL: find and remove data of the wrong type in our data frames

char in numeric columns, numbers in char columns

In [1]:
import numpy as np
import pandas as pd
import random

## Data Creation

In [2]:
df = pd.DataFrame({'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7),
                   'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6)})

In [3]:
df.head(7)

,num_col,name_col
0,3,bob
1,5.46,sue
2,62.1,steve
3,nan,24.5
4,False,kim
5,angie,True
6,3,nan


In [4]:
df.dtypes

num_col     object
name_col    object
dtype: object

## Remove Chars and Booleans from a Numeric column (but keep NaNs for later review)

In [5]:
#functionalized.  usage: char_finder(df, 'num_col')
def char_finder(data_frame, series_name):
    cnt=0
    print(series_name)
    for row in data_frame[series_name]:
        try:
            float(row) #changed to float to not flag NaNs or decimals.  
            pass
        except ValueError:
            print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
        cnt+=1

In [6]:
char_finder(df, 'num_col')

num_col
False -> at row:4
angie -> at row:5
False -> at row:10
angie -> at row:11
False -> at row:16
angie -> at row:17
False -> at row:22
angie -> at row:23
False -> at row:28
angie -> at row:29
False -> at row:34
angie -> at row:35
False -> at row:40
angie -> at row:41


In [7]:
df.dtypes['num_col']

dtype('O')

In [8]:
#functionalized.  usage: char_finder(df, 'num_col')
def char_fixer(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            float(row) #changed to float to not flag NaNs or decimals.  
            pass
        except ValueError:
            data_frame.drop([cnt], inplace=True) #or assign to some other desired value
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('float64', errors='raise')
    data_frame.reset_index(drop=True, inplace=True)

In [9]:
char_fixer(df,'num_col')

Let's check to see what type our column is now

In [10]:
df.dtypes['num_col']

dtype('float64')

We can use the finder function to do one last check to make sure the characters are removed

In [11]:
char_finder(df, 'num_col')

num_col


## Remove numbers and booleans from a character column

In [12]:
df = pd.DataFrame({'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7),
                   'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6)})

Keep in mind that Pandas cast booleans to strings in the dataframe conversion.

In [13]:
df['name_col'][5]

'True'

In [14]:
isinstance(df['name_col'][5], bool)

False

Therefore, float() will not be able to separate the booleans from the character values, and we'll have to deal with them separately

In [15]:
#functionalized.  usage: num_finder(df, 'name_col')
def num_finder(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            int(float(row)) # if no error, report found numeric in bottom 'else:'
        except ValueError:
            if row=='True' or row=='False':
                # report the True or False entries
                print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
            else: #Chars or NaNs
                pass
        else:
            print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
        cnt+=1

In [16]:
num_finder(df,'name_col')

24.5 -> at row:3
True -> at row:5
24.5 -> at row:10
True -> at row:12
24.5 -> at row:17
True -> at row:19
24.5 -> at row:24
True -> at row:26
24.5 -> at row:31
True -> at row:33
24.5 -> at row:38
True -> at row:40


In [32]:
#functionalized.  usage: num_finder(df, 'name_col')
def num_fixer(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            int(float(row)) # if no error, drop it in the bottom else:'
        except ValueError:
            if row=='True' or row=='False': #drop if would-be boolean
                data_frame.drop([cnt], inplace=True) 
            else: #Chars or NaNs
                pass
        else:
            data_frame.drop([cnt], inplace=True) 
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('string', errors='raise')
    data_frame.reset_index(drop=True, inplace=True)

In [18]:
num_fixer(df,'name_col')

In [34]:
char_fixer(df,'num_col')

In [36]:
df.head(10)

,num_col,name_col
0,3.00,bob
1,5.46,sue
2,62.10,steve
3,3.00,nan
4,5.46,bob
5,62.10,sue
6,NaN,steve
7,5.46,nan
8,62.10,bob
9,NaN,sue


In [37]:
df.dtypes

num_col     float64
name_col     string
dtype: object

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num_col   15 non-null     float64
 1   name_col  20 non-null     string 
dtypes: float64(1), string(1)
memory usage: 448.0 bytes


## Speed of functions

Testing on fixer function as %timeit runs 1000s of loops and finder functions print out data

In [67]:
dfbig = pd.DataFrame({'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6000),
                      'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7000)})

In [64]:
dfbig.shape

(42000, 2)

In [65]:
%timeit  char_fixer(dfbig,'num_col')

9.74 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Try to be faster by using pandas apply() instead

In [66]:
def char_by_line_fixer(element):
    try:
        float(element) #changed to float to not flag NaNs or decimals.  Adjust as necessary
        return element
    except ValueError:
        return np.nan

In [ ]:
dfbig['num_col'] = dfbig['num_col'].apply(char_by_line_fixer)

In [68]:
%timeit dfbig['num_col'] = dfbig['num_col'].apply(char_by_line_fixer)

22.6 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
char_finder(dfbig, 'num_col')

num_col


In [29]:
dfbig.head(7)

,name_col,num_col
0,bob,3.00
1,sue,5.46
2,steve,62.10
3,24.5,NaN
4,nan,3.00
5,bob,5.46
6,sue,62.10
